<div style="background-color:#000;"><img src="pqn.png"></img></div>

These libraries provide tools for data manipulation, numerical operations, visualization, and financial data retrieval

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import yfinance as yf

### Retrieve option data

We start by fetching option data for a specific ticker and expiration date using the yfinance library.

In [2]:
ticker = "QQQ"
expiration_date = "2025-02-28"

In [3]:
stock = yf.Ticker(ticker)
options = stock.option_chain(expiration_date)
calls = options.calls

We use yfinance to get option data for the QQQ ticker with an expiration date of February 28, 2025. We retrieve the full option chain and then isolate the call options. This gives us a dataset of call options to work with for our analysis.

### Calculate option gamma

Next, we define a function to calculate the gamma of a call option and apply it to our dataset.

In [ ]:
def call_gamma(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return norm.pdf(d1) / (S * sigma * np.sqrt(T))

In [ ]:
S = stock.info["navPrice"] # navPrice for ETFs, currentPrice for for stocks
r = 0.0423  # Risk-free rate
T = ((pd.to_datetime(expiration_date) - pd.Timestamp.now()).days + 1) / 365

In [ ]:
atm_strike = (
    calls
    .strike[calls.strike >= S]
    .sort_values()
    .iloc[0]
)

In [ ]:
calls['gamma'] = (
    calls
    .apply(lambda row: call_gamma(S, row['strike'], T, r, row['impliedVolatility']), axis=1)
)

We define a function to calculate the gamma of a call option using the Black-Scholes formula. We then set up our parameters, including the current price, risk-free rate, and time to expiration. We find the at-the-money strike price and calculate the gamma for each call option in our dataset.

### Prepare data for visualization

We filter our data to focus on options near the current price and select the relevant columns for our analysis.

In [ ]:
gamma = (
    calls[
        (calls.strike >= atm_strike * 0.95) & (calls.strike <= atm_strike * 1.05)
    ][[
        "strike", 
        "impliedVolatility", 
        "openInterest", 
        "gamma"
    ]]
)

We create a new dataframe called 'gamma' that contains options with strike prices within 5% of the at-the-money strike. We select the strike price, implied volatility, open interest, and calculated gamma for each of these options. This gives us a focused dataset for our visualization.

### Visualize gamma distribution

Finally, we create a plot to visualize the gamma distribution and open interest for our selected options.

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.plot(gamma['strike'], gamma['gamma'], label='Total Gamma', color='b')
ax1.axvline(S, color='r', linestyle='--', label='Current Price')
ax1.set_xlabel('Strike Price')
ax1.set_ylabel('Gamma', color='b')
ax1.tick_params(axis='y', labelcolor='b')

ax1.set_ylim(0.00, ax1.get_ylim()[1])

ax2 = ax1.twinx()
ax2.bar(gamma['strike'], gamma['openInterest'], alpha=0.3, color='gray', label='Open Interest')
ax2.set_ylabel('Open Interest', color='gray')
ax2.tick_params(axis='y', labelcolor='gray')

plt.title(f'Gamma Distribution for {ticker} Options')
fig.legend(loc="upper left", bbox_to_anchor=(0.1,0.9))
ax1.grid(True)

plt.show()

We create a plot with two y-axes. The primary axis shows the gamma distribution as a blue line, with a red vertical line indicating the current

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.